In [1]:
#import Pkg; Pkg.add("DataArrays")
#Pkg.add("CSV")
#Pkg.add("StatsBase")

using LowRankModels
using CSV
using DataFrames
using Pkg
using StatsBase
using Random, SparseArrays
#using DataArrays

In [2]:
df = CSV.read("../data/PATIENTS.csv")

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP
,Int64,Int64,String,String,String?,String?
1,234,249,F,2075-03-13 00:00:00,missing,missing
2,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00
3,236,251,M,2090-03-15 00:00:00,missing,missing
4,237,252,M,2078-03-06 00:00:00,missing,missing
5,238,253,F,2089-11-26 00:00:00,missing,missing
6,239,255,M,2109-08-05 00:00:00,missing,missing
7,240,256,M,2086-07-31 00:00:00,missing,missing
8,241,257,F,2031-04-03 00:00:00,2121-07-08 00:00:00,2121-07-08 00:00:00
9,242,258,F,2124-09-19 00:00:00,missing,missing


In [3]:
imp = CSV.read("../data/Random_Forest_Importance.csv")
impp = CSV.read("../data/Positive_Random_Forest_Importance.csv")

hyper = CSV.read("../data/Hypertension_Matrix.csv")
hyper = select!(hyper, Not(:Column1))
hyper = select!(hyper, Not(:SUBJECT_ID )) 
hyper

,003,004,005,007,008,009,010,011,012,013,014,015
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
#drop columns
#df3 = select!(df3, Not(:Column1))
df3 = CSV.read("../data/Diabetes_Numerical.csv")
df3 = select!(df3, Not(:SUBJECT_ID )) 
df3

,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3.666,1,4,0,1,0,0,0
2,1.2885,1,6,0,1,0,0,0
3,1.0508,1,8,1,0,0,0,0
4,14.2664,2,9,0,1,0,0,0
5,0.5124,1,4,0,1,0,0,0
6,3.5466,1,6,0,1,0,0,0
7,1.1224,1,8,1,0,0,0,0
8,5.3757,1,7,1,0,0,0,0
9,1.1397,1,7,0,1,0,0,0


In [5]:
diabetesdrg = CSV.read("../data/diabetesdrg.csv")
diabetesdrg = select!(diabetesdrg, Not(:SUBJECT_ID )) 
diabetesdrg

,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,14.2664,2,9,0,1,0,0,0
2,5.3757,1,7,1,0,0,0,0
3,1.9138,1,7,0,1,0,0,0
4,2.2017,1,9,0,1,0,0,0
5,1.2469,1,8,1,0,0,0,0
6,5.1326,1,8,0,1,0,0,0
7,6.1813,2,9,0,1,0,0,0
8,3.7078,1,8,0,1,0,0,0
9,26.919,7,6,0,1,0,0,0


#column names
"""
for name in names(df3)
       print(":", name, ", ")
   end
"""
#describe(df3)

# K-means with quadloss

In [6]:
# minimize_{X<=0} ||A - XY||^2
function kmeans_quadloss_drg(kval, matrix)
    k = kval
    A = matrix
    loss = QuadLoss()
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,loss,rx,ry,k)
    
    X,Y,ch = fit!(glrm)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xquaddrg", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
    
    #imputey = impute(loss, Y) 
    #imputey  = convert(DataFrame, imputey) 
    #CSV.write(string("../data/iyquad", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    return A,X,Y,ch
end

kmeans_quadloss_drg (generic function with 1 method)

In [12]:
kmeans_quadloss_drg(3, diabetesdrg)

Fitting GLRM
Iteration 10: objective value = 1.065642621811795e6


(7901×922 DataFrame. Omitted printing of 917 columns
│ Row  │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│      │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├──────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1    │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 2    │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 3    │ 1.9138  │ 1               │ 7           │ 0        │ 1         │
│ 4    │ 2.2017  │ 1               │ 9           │ 0        │ 1         │
│ 5    │ 1.2469  │ 1               │ 8           │ 1        │ 0         │
│ 6    │ 5.1326  │ 1               │ 8           │ 0        │ 1         │
│ 7    │ 6.1813  │ 2               │ 9           │ 0        │ 1         │
│ 8    │ 3.7078  │ 1               │ 8           │ 0        │ 1         │
│ 9    │ 26.919  │ 7               │ 6           │ 0        │ 1         │
│ 10   │ 10.1558 │ 2               │ 8           │ 0       

In [5]:
# minimize_{X<=0} ||A - XY||^2
function kmeans_quadloss(kval, matrix)
    k = kval
    A = matrix
    loss = QuadLoss()
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,loss,rx,ry,k)
    
    X,Y,ch = fit!(glrm)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xquad", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
    
    #imputey = impute(loss, Y) 
    #imputey  = convert(DataFrame, imputey) 
    #CSV.write(string("../data/iyquad", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    return A,X,Y,ch
end

kmeans_quadloss (generic function with 1 method)

In [23]:
kmeans_quadloss(3, df3)

Fitting GLRM
Iteration 10: objective value = 1.3481035985825742e6


(10310×920 DataFrame. Omitted printing of 915 columns
│ Row   │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│       │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├───────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1     │ 3.666   │ 1               │ 4           │ 0        │ 1         │
│ 2     │ 1.2885  │ 1               │ 6           │ 0        │ 1         │
│ 3     │ 1.0508  │ 1               │ 8           │ 1        │ 0         │
│ 4     │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 5     │ 0.5124  │ 1               │ 4           │ 0        │ 1         │
│ 6     │ 3.5466  │ 1               │ 6           │ 0        │ 1         │
│ 7     │ 1.1224  │ 1               │ 8           │ 1        │ 0         │
│ 8     │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 9     │ 1.1397  │ 1               │ 7           │ 0        │ 1         │
│ 10    │ 1.9138  │ 1               │ 7       

In [22]:
kmeans_quadloss(4, df3)

Fitting GLRM
Iteration 10: objective value = 1.348808320954107e6


(10310×920 DataFrame. Omitted printing of 915 columns
│ Row   │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│       │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├───────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1     │ 3.666   │ 1               │ 4           │ 0        │ 1         │
│ 2     │ 1.2885  │ 1               │ 6           │ 0        │ 1         │
│ 3     │ 1.0508  │ 1               │ 8           │ 1        │ 0         │
│ 4     │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 5     │ 0.5124  │ 1               │ 4           │ 0        │ 1         │
│ 6     │ 3.5466  │ 1               │ 6           │ 0        │ 1         │
│ 7     │ 1.1224  │ 1               │ 8           │ 1        │ 0         │
│ 8     │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 9     │ 1.1397  │ 1               │ 7           │ 0        │ 1         │
│ 10    │ 1.9138  │ 1               │ 7       

In [21]:
kmeans_quadloss(5, df3)

Fitting GLRM
Iteration 10: objective value = 1.3316886178968598e6


(10310×920 DataFrame. Omitted printing of 915 columns
│ Row   │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│       │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├───────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1     │ 3.666   │ 1               │ 4           │ 0        │ 1         │
│ 2     │ 1.2885  │ 1               │ 6           │ 0        │ 1         │
│ 3     │ 1.0508  │ 1               │ 8           │ 1        │ 0         │
│ 4     │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 5     │ 0.5124  │ 1               │ 4           │ 0        │ 1         │
│ 6     │ 3.5466  │ 1               │ 6           │ 0        │ 1         │
│ 7     │ 1.1224  │ 1               │ 8           │ 1        │ 0         │
│ 8     │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 9     │ 1.1397  │ 1               │ 7           │ 0        │ 1         │
│ 10    │ 1.9138  │ 1               │ 7       

In [20]:
kmeans_quadloss(6, df3)

Fitting GLRM
Iteration 10: objective value = 1.269579828183824e6


(10310×920 DataFrame. Omitted printing of 915 columns
│ Row   │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│       │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├───────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1     │ 3.666   │ 1               │ 4           │ 0        │ 1         │
│ 2     │ 1.2885  │ 1               │ 6           │ 0        │ 1         │
│ 3     │ 1.0508  │ 1               │ 8           │ 1        │ 0         │
│ 4     │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 5     │ 0.5124  │ 1               │ 4           │ 0        │ 1         │
│ 6     │ 3.5466  │ 1               │ 6           │ 0        │ 1         │
│ 7     │ 1.1224  │ 1               │ 8           │ 1        │ 0         │
│ 8     │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 9     │ 1.1397  │ 1               │ 7           │ 0        │ 1         │
│ 10    │ 1.9138  │ 1               │ 7       

# K-means different loss functions and proximal gradient descent ( i = 40)

In [13]:
function kmeans_proxgrad_drg(kval, lossarr, matrix)
    k = kval
    losses = lossarr
    A = matrix
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,losses,rx,ry,k)
    params = ProxGradParams(inner_iter_Y=40)

    
    X,Y,ch = fit!(glrm, params) # objective value
    println("X: ")
    println(X)
    println("Y: ")
    println(Y)
    
    imputey = impute(losses, Y) 
    imputey  = convert(DataFrame, imputey) 
    CSV.write(string("../data/iyproxgrad_drg", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xproxgrad_drg", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
end    

kmeans_proxgrad_drg (generic function with 1 method)

In [14]:
#loss function array
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
loss4 = fill(OrdinalHingeLoss(), 2)

lossarr = [loss1; loss3; loss2; loss4]
lossarr

922-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [15]:
kmeans_proxgrad_drg(3, lossarr, diabetesdrg)

Fitting GLRM
Iteration 10: objective value = 393183.25394504494
X: 


[0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0

 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0

 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0

 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0

[3.0288834067096273 1.2462431020415239 6.9995097561832305 -1.030348059956518 1.0323061437485725 -1.1137848365393042 -1.0064801245215333 -1.2844365265266644 -1.119385526319113 1.0003977170752159 -1.0021378988208178 -1.0000079598974163 -1.0000353247904572 -1.462786959395701 -1.0000045388272492 -1.0000100458946435 -1.000045852198012 -1.0067646320093333 -1.3201907960146833 -1.0035140468327561 -0.5185155997927121 -1.9564921027875715 -1.043471762418985 -1.1650881648990563 -1.2894095225843047 -1.1198545075261317 -1.1706191246675965 -1.1579335225806409 -1.1136727782258633 -1.2321200184796095 -1.0000021445051963 -1.7351969725896772 -1.1962246130501228 -1.188685601159026 -1.0000046393097695 -1.12268195185525 -1.376842857102039 -1.0154826141704503 -1.003114107762241 -1.0593052904094156 -1.0000022613638737 -1.000008699781102 -1.007733739349522 -1.301740126483424 -1.5473930325428968 -1.1626214576318574 -1.0000012794968236 -1.6504825051532808 -1.336103644318694 -1.355170767000147 -1.2222311867221771

"../data/Xproxgrad_drg3.csv"

In [ ]:
function kmeans_proxgrad(kval, lossarr, matrix)
    k = kval
    losses = lossarr
    A = matrix
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,losses,rx,ry,k)
    params = ProxGradParams(inner_iter_Y=40)

    
    X,Y,ch = fit!(glrm, params) # objective value
    println("X: ")
    println(X)
    println("Y: ")
    println(Y)
    
    imputey = impute(losses, Y) 
    imputey  = convert(DataFrame, imputey) 
    CSV.write(string("../data/iyproxgrad", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xproxgrad", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
end   

In [11]:
#loss function array
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
lossarr = [loss1; loss3; loss2]
lossarr

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [ ]:
kmeans_proxgrad(3, lossarr, df3)

In [12]:
kmeans_proxgrad(4, lossarr, df3)

Fitting GLRM
Iteration 10: objective value = 456662.5980351248
X: 


[0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0

 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0

 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0

...

 -1.0000009663595082 -1.0000012295040672 -1.1737931948410196 -1.0226164405932354 -1.0247027858802176 -1.1182124246598426 -1.19148395654122 -1.1580649065878437 -1.1048043471296873 -1.1289135721478334 -1.168507640374287 -1.000001205278964 -1.0636118076822854 -1.1981519242967646 -1.0636020756529028 -1.0260789593008948 -1.0000015479132074 -1.319478353525245 -1.0000016224061727 -1.4528074418380001 -1.1331923906676076 -1.0030008202864857 -1.1853899781325032 -1.069911257519805 -1.322933984816195 -1.2706013069327136 -1.0425743778272683 -1.162302646340657 -1.2805895279995578 -1.242355039367592 -1.0481395488475456 -1.0000007343190747 -1.1879672901817453 -1.1484396999106463 -1.0000262877979644 0.9945806846427664 -1.0162670780080705 -1.0002675955490783 -1.0394011610248373 -1.0500916274787264 -1.0026140557587062 -1.1351172796506985 -1.000045014328688 -0.9999760846668873 -0.21200877880508237 -1.169797320966936 -1.104505497685213 -1.1549491229944402 -1.0240923277952896 -1.4983115561408868 -1.19122557

 -1.2258721271449984 -1.084294265700183 -1.169221540831931 -1.1133426734637801 -1.0040993591962197 -1.073266571572824 -1.0000004772979676 -1.0490633610130558 -1.1775342284775774 -1.0865471231531778 -1.0848214208210853 -1.2220912313372654 -1.0522344835936523 -1.0000036678120916 -1.0560509272461593 -1.3133316530066026 -1.6270084022159306 -1.0292767029777632 -1.0120354443895148 -1.0887019727392389 -1.284643724667236 -1.1156047127729425 -1.1889424367242953 -1.0000015965202143 -1.2573529804399604 -1.0000004584074378 -1.169973618134335 -1.2193070863228432 -1.000000369362939 -0.9999974962743631 -0.999999839657204 -1.0077276528286658 -1.0397623165307768 -1.1056322002949575 -1.0238301371747762 -1.0000002710438232 -1.2722068164357216 -1.464676610714085 -1.1125654086499503 -1.000000388574677 -1.1894827327423965 -1.0000006206909544 -1.0851982178041992 -1.258621190691534 -1.294852141345816 -1.060917598835593 -1.1897125178077168 -1.8987460122195563 -1.0000014692899875 -1.0109882020498164 -1.25229974

 -1.2525158345743366 -1.0000019007048675 -1.0287876527433155 -0.9999702253265713 -1.000005396797625 -1.0514696081358639 -1.1708412898978147 -1.3672280864872624 -1.2754551035021868 -0.9999284065058867 -1.2105144725802703 -1.209317480804135 -0.9999956656162248 -0.999999852138337 -1.3814455553616272 -1.1423977578417577 -1.1384049724252845 -1.0000007811696938 -1.2198324619792533 -1.1388414608104842 -1.0295057940573504 -1.0005616506346973 -0.9999985810325962 -1.1937448629050784 -1.000006487526822 -1.0000015568782936 -1.0678659062057232 -1.0000098196673604 -1.1133843503392398 -1.00235856078847 -1.0907199758200372 -1.3312835169058548 -1.1746260519853946 -1.1413638939035886 -1.1559783238025598 -1.1363712730700772 -1.3051073443591275 -1.0178716424781888 -1.1384919213640288 -1.9239891463128882 -1.0275789714190076 -1.0000049012144119 -1.0000581188997162 -1.2072853083245294 -1.000001854143504 -1.2018663433426553 -1.0201236845107875 -1.1152514592835154 -0.999966152378885 -1.1201865076612552 -1.0000

 -1.8982686433490632 -1.187301370866229 -1.0062592794712126 -2.040837562469066 -1.6625549495193856 -2.157380574863025 -1.0000002021744905 -1.150738822921557 -1.2136845738871838 -1.2159333817506797 -1.1937823495866149 -1.0735727065198242 -1.2333535036347767 -1.1481416900794714 -1.073298528148493 -1.6427377054351395]


"../data/Xproxgrad4.csv"

In [13]:
kmeans_proxgrad(5, lossarr, df3)

Fitting GLRM
Iteration 10: objective value = 457255.180086002
X: 


[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0

 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0

...

 -1.0129318390663458 -1.5630040310223385 -1.116821247121279 -1.1495597184446942 -1.124928504217257 -1.0000017811459796 -1.638359962660944 -1.0000932245722385 -1.0333673668817136 -1.1494287374404721 -1.0882535194733614 -3.7424307416392804 -0.9999744420332427 -1.0830234448160925 -1.0000005671064292 -1.10119976524468 -1.0000140983180612 -1.000000290646085 -1.1891230822197172 -1.0049601266265158 -1.1470861377477495 -1.240537808400243 -1.5856471941454102 -1.0000011870112855 -1.5866543108244413 -1.0000001385107664 -1.0962021993327176 -1.0000025113026905 -1.0882407684320072 -1.099568411497265 -1.0000036085533692 -2.67128503887718 -1.2042263653370935 -1.0955220441050422 -1.0605059446032645 -1.016578208841266 -0.999999996676354 -2.1480901082053405 -1.1413547026062867 -1.123837416233365 -1.024874429810601 -1.0341810788628707 -1.008265303842003 -1.0174513271574503 -2.2490466642148603 0.9991948852739582 -1.0242181091899845 -1.0008098407060093 -1.0114117218711451 -1.0089558534558758 -1.003936590372

 -2.1398956613232514 -1.0892680173737765 -1.0032213524702156 -1.0987434420378095 -1.0087288184602117 -1.2015259325150047 -1.0000003887539712 -1.031128392133175 -1.0000017028109738 -1.1023257752979374 -1.000000218423979 -1.0877942428182923 -1.0008602534942734 -1.024942920740552 -1.0333788836597622 -1.193603319019828 -1.0052311912446399 -1.2065464574771014 -1.1356293852544244 -1.0000008685166453 -0.9999904285903781 -1.000000686622565 -1.1821250469338656 -1.0000021553573497 -1.0358227701309055 -1.126250315960113 -1.0200924174342114 -1.1106228179438442 -1.1054205279052363 -1.0667447911669425 -1.1898340358154356 -1.0000047904552096 -1.1741267826674517 -1.0000028825077696 -1.0551803278694174 -1.111824494429276 -1.4551890938038856 -1.0000008223808898 -1.3137048106097695 -1.000005169470182 -1.11559336816816 -1.2142179856345094 -1.619368668763815 -1.1347446874903138 -1.1370610548957256 -1.0550943432939721 -1.0000001994044607 -1.3296199609402675 -1.1210641001282973 -1.0229340611120814 -1.0508564

 -1.0000083520247316 -0.9999657812691433 -1.0000074019239478 -1.1240188948926766 -1.007649128292019 -1.1599224711899854 -1.026680740177231 -1.0000004184984372 -1.016484444788092 -0.9999995869665421 -1.0754115940597058 -1.0079809301920348 -1.0079426993998948 -1.151157045380367 -1.194966589326885 -1.0320553417664604 -1.1867178154118994 -0.9999970936203249 -1.0486111971092524 -0.9998815794798944 -1.1149642752067022 -1.0084690082934917 -1.0294451160849862 -1.0000048667352628 -1.000000466024701 -1.1581827354288454 -1.000001474754224 -1.256851815638479 -1.0000009156731497 -1.1907950984697198 -1.0183615459284885 -0.9998086674416787 -1.2918306778099404 -1.0156195303745639 -1.027159873181067 -1.044727237878114 -1.07418980390922 -1.1455139402616255 -1.0270932226950011 -1.0939151145924457 -1.0124835939244343 -1.148150290430443 -1.0509941246891106 -1.0476923993051748 -1.6791085563896255 -1.0842261573839718 -1.034927643806495 -1.1589919740959285 -1.0048344314519007 -1.035217238319812 -1.10934676279

 -1.0802327750895993 -1.0000011372533637 -1.147095243919213 -1.1458947342096009 -1.0000038398226494 -1.0000017918512407 -1.0573441677124396 -1.0000027729752412 -1.0000014718694918 -1.3103650478114206 -1.2734073046873438 -1.0219303242119702 -0.9999998432507821 -1.5913880004433592 -1.2238433234041275 -1.0000009912035182 -1.1060756780410406 -1.1024186397403852 -1.062297429488254 -1.7756290178365242 -1.0660206960832084 -1.3112358195227858 -1.0000008970502572 -1.0544392584050162 -1.1540139725812435 -1.0155651459268238]


"../data/Xproxgrad5.csv"

In [ ]:
kmeans_proxgrad(6, lossarr, df3)

In [7]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]

losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [25]:

k = 7
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm7 = GLRM(A,losses,rx,ry,k)
params7 = ProxGradParams(inner_iter_Y=40)
X7,Y7,ch7 = fit!(glrm7, params7)


Fitting GLRM
Iteration 10: objective value = 455586.3300457709


([0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [2.891401019255788 1.284647977777654 … -1.0000003485855393 -1.201684890925261; 3.0472118483057886 1.3019132138356035 … -1.054510468064438 -1.3203688977059513; … ; 3.0014550786000904 1.2768594556466304 … -1.1530029427806434 -1.0655229446433998; 3.042416092591336 1.2506347420467852 … -1.0002717324856651 -1.1643981186670789], ConvergenceHistory("ProxGradGLRM", [Inf, 456816.440335322, 455663.05238130275, 455619.15154361044, 455604.71384979086, 455599.1661339887, 455595.5450104161, 455592.62139858556, 455590.0949900498, 455588.0714520461, 455586.3300457709, 455584.80367816816], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 211.11720514297485, 449.2978570461273, 726.6846570968628, 984.5985560417175, 1234.489140033722, 1481.4453201293945, 1727.8016421794891, 1974.2346062660217, 2224.085343360901, 2475.9831013679504, 2

In [10]:
X7

7×10310 Array{Float64,2}:
 0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  1.0  1.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [11]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm7, nfolds=2, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 4742124 samples and testing on 4743076
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 226781.71437389767
computing train and test error for fold 1:
	train error: 0.04782224217856629
	test error:  0.04813035228638062

forming train and test GLRM for fold 2
training model on 4743076 samples and testing on 4742124
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 228273.7717253448
computing train and test error for fold 2:
	train error: 0.04812738528291502
	test error:  0.04782559310411637


,train_error,test_error
,Float64,Float64
1,0.0478222,0.0481304
2,0.0481274,0.0478256


In [12]:
hingek7 = convert(DataFrame, X7) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [13]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingeparam100.7.csv",  hingek7)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingeparam100.7.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [29]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingeparam7.csv",  hingek7)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingeparam7.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
#k=6

In [25]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [26]:
k = 6
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm640 = GLRM(A,losses,rx,ry,k)
params640 = ProxGradParams(inner_iter_Y=40)
X640,Y640,ch640 = fit!(glrm640, params640)


Fitting GLRM
Iteration 10: objective value = 458147.6334159647


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [2.862139646112679 1.2882879242529348 … -1.4142629657705146 -1.0953637622419514; 3.120776367275652 1.321049766273326 … -1.0094524446793753 -1.0795313884425397; … ; 3.0049082038353023 1.2792889112368235 … -1.1812218381940358 -1.325774051714759; 3.019549514314535 1.2672591030708074 … -1.0458303459879459 -1.0459889046099078], ConvergenceHistory("ProxGradGLRM", [Inf, 458761.2946550906, 458222.3481236272, 458186.0534051756, 458173.4583447895, 458165.5477371825, 458159.4926408788, 458155.17257049057, 458151.9948982341, 458149.5871506245, 458147.6334159647, 458146.0088561262], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 227.18541502952576, 482.83217000961304, 737.7277908325195, 1094.9590647220612, 1361.526443719864, 1637.1732137203217, 1887.304390668869, 2192.9811835289, 2474.447030544281, 2737.5539355278015, 3042.6

In [27]:
X640

6×10310 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  1.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     1.0  0.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [28]:
println("Computing training and testing error as a function of iteration number")
train_error, test_error = cv_by_iter(glrm640)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing training and testing error as a function of iteration number
 train error  test error        time
  4.0928e+05  4.7922e-02  1.1685e+02


,train_error,test_error
,Float64,Float64
1,409279.0,0.0479225


In [29]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm640, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 7112747 samples and testing on 2372453
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 343988.0881832769
computing train and test error for fold 1:
	train error: 0.048361706353199606
	test error:  0.04809075449192487

forming train and test GLRM for fold 2
training model on 7115248 samples and testing on 2369952
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 343450.4746718845
computing train and test error for fold 2:
	train error: 0.04826922832553974
	test error:  0.048368865825785305

forming train and test GLRM for fold 3
training model on 7115369 samples and testing on 2369831
fitting train GLRM for fold 3
Fitting GLRM
Iteration 10: objective value = 343848.67574424023
computing train and test error for fold 3:
	train error: 0.04832435294669485
	test error:  0.04820023153056868

for

,train_error,test_error
,Float64,Float64
1,0.0483617,0.0480908
2,0.0482692,0.0483689
3,0.0483244,0.0482002
4,0.0482166,0.0485213


In [30]:
X640 = convert(DataFrame, X640) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
CSV.write("/Users/kathy908000/github/health-care-research/data/X640.csv",  X640)
CSV.read("/Users/kathy908000/github/health-care-research/data/X640.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
Y640

6×920 Array{Float64,2}:
 2.86214  1.28829  7.0      -1.0186    …  -1.22457  -1.41426  -1.09536
 3.12078  1.32105  7.00912  -0.999717     -1.08495  -1.00945  -1.07953
 3.03042  1.28319  7.03633  -1.00014      -1.0      -1.0      -1.15889
 2.94126  1.23686  7.10748  -1.00195      -1.12952  -1.07489  -1.1381
 3.00491  1.27929  7.00035  -1.06292      -1.01068  -1.18122  -1.32577
 3.01955  1.26726  7.00253  -0.999951  …  -1.03943  -1.04583  -1.04599

In [36]:
imputey640 = impute(losses, Y640) 
imputey640

6×920 Array{Number,2}:
 2.86214  1.28829  7  false  true  …  false  false  false  false  false
 3.12078  1.32105  7  false  true     false  false  false  false  false
 3.03042  1.28319  7  false  true     false  false  false  false  false
 2.94126  1.23686  7  false  true     false  false  false  false  false
 3.00491  1.27929  7  false  true     false  false  false  false  false
 3.01955  1.26726  7  false  true  …  false  false  false  false  false

In [45]:
#imputey640[1, :]
print(imputey640[1, :])

Number[2.862139646112679, 1.2882879242529348, 7, false, true, false, false, false, false, true, false, false, false, false, true, false, false, false, false, false, true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, fal

In [47]:
imputey640[1, :] == imputey640[2, :]

false

In [27]:
#k=5

In [10]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [ ]:
k = 5
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm540 = GLRM(A,losses,rx,ry,k)
params540 = ProxGradParams(inner_iter_Y=40)
X540,Y540,ch540 = fit!(glrm540, params540)

In [ ]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm540, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)



In [ ]:
X540 = convert(DataFrame, X540) 

In [ ]:
CSV.write("/Users/kathy908000/github/health-care-research/data/X540.csv",  X540)
CSV.read("/Users/kathy908000/github/health-care-research/data/X540.csv")


In [ ]:
Y540

In [ ]:
imputey540 = impute(losses, Y540) 
CSV.write("/Users/kathy908000/github/health-care-research/data/iy540.csv",  imputey540)
CSV.read("/Users/kathy908000/github/health-care-research/data/iy540.csv")

In [ ]:
#k=4 original  (did not run)

In [ ]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

In [ ]:
k = 4
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm440 = GLRM(A,losses,rx,ry,k)
params440 = ProxGradParams(inner_iter_Y=40)

In [ ]:
X440,Y440,ch440 = fit!(glrm440, params440)

In [ ]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm440, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

In [ ]:
X440 = convert(DataFrame, X440) 
CSV.write("/Users/kathy908000/github/health-care-research/data/X440.csv",  X440)
CSV.read("/Users/kathy908000/github/health-care-research/data/X440.csv")


In [ ]:
Y440

In [ ]:

imputey440 = impute(losses, Y440) 
CSV.write("/Users/kathy908000/github/health-care-research/data/iy440.csv",  imputey440)
CSV.read("/Users/kathy908000/github/health-care-research/data/iy440.csv")

In [ ]:
#k=3

In [4]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [6]:
k = 3
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm340 = GLRM(A,losses,rx,ry,k)
params340 = ProxGradParams(inner_iter_Y=40)


ProxGradParams(1.0, 100, 1, 40, 1.0e-5, 0.0001, 0.01)

In [7]:
X340,Y340,ch340 = fit!(glrm340, params340)

Fitting GLRM
Iteration 10: objective value = 460455.0771161948


([0.0 1.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0; 1.0 0.0 … 0.0 0.0], [3.147805880019738 1.2376816867033162 … -1.15905630764086 -1.383198064261709; 2.871124633823481 1.2444539172584015 … -1.239819813135892 -1.291888072071956; 3.0523080928026833 1.2614522511491024 … -1.0000080851156525 -1.0408613450497213], ConvergenceHistory("ProxGradGLRM", [Inf, 462137.940573622, 461364.56644663285, 460970.7127084627, 460521.45564141974, 460501.43166199565, 460485.6681927925, 460474.10536701046, 460465.70805219136, 460459.6498293013, 460455.0771161948, 460451.4619882424], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 171.87946009635925, 372.6581370830536, 572.042307138443, 775.2450821399689, 995.1492972373962, 1228.0673682689667, 1455.9715762138367, 1688.2711172103882, 1904.46892619133, 2117.194132089615, 2324.4776699543], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0))

In [8]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm340, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 7112389 samples and testing on 2372811
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 345467.09580262064
computing train and test error for fold 1:
	train error: 0.04857212751618507
	test error:  0.048407673083230005

forming train and test GLRM for fold 2
training model on 7115593 samples and testing on 2369607
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 344412.3265595404
computing train and test error for fold 2:
	train error: 0.04840209799381357
	test error:  0.04892117592870752

forming train and test GLRM for fold 3
training model on 7115171 samples and testing on 2370029
fitting train GLRM for fold 3
Fitting GLRM
Iteration 10: objective value = 345790.48890368384
computing train and test error for fold 3:
	train error: 0.04859847015262049
	test error:  0.048334133182781645

fo

,train_error,test_error
,Float64,Float64
1,0.0485721,0.0484077
2,0.0484021,0.0489212
3,0.0485985,0.0483341
4,0.0485515,0.0484703


In [9]:
X340 = convert(DataFrame, X340) 
CSV.write("/Users/kathy908000/github/health-care-research/data/X340.csv",  X340)
CSV.read("/Users/kathy908000/github/health-care-research/data/X340.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [10]:
Y340

3×920 Array{Float64,2}:
 3.14781  1.23768  7.08981  -1.00192  …  -1.0      -1.15906  -1.3832
 2.87112  1.24445  6.9996   -1.00003     -1.01627  -1.23982  -1.29189
 3.05231  1.26145  7.29001  -1.01071     -1.10831  -1.00001  -1.04086

In [12]:
imputey340 = impute(losses, Y340) 

3×920 Array{Number,2}:
 3.14781  1.23768  7  false  true  …  false  false  false  false  false
 2.87112  1.24445  7  false  true     false  false  false  false  false
 3.05231  1.26145  7  false  true     false  false  false  false  false

In [13]:
imputey340d  = convert(DataFrame, imputey340) 
CSV.write("/Users/kathy908000/github/health-care-research/data/iy340.csv",  imputey340d)
CSV.read("/Users/kathy908000/github/health-care-research/data/iy340.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
,Float64,Float64,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,3.14781,1.23768,7,0,1,0,0,0,0,1,0,0,0
2,2.87112,1.24445,7,0,1,0,0,0,0,1,0,0,0
3,3.05231,1.26145,7,0,1,0,0,0,0,1,0,0,0


In [ ]:
#k=4 with output but inner_iter_Y=100

In [20]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [21]:
k = 4
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm04 = GLRM(A,losses,rx,ry,k)
params04 = ProxGradParams(inner_iter_Y=100)
X04,Y04,ch04 = fit!(glrm04)

Fitting GLRM
Iteration 10: objective value = 537407.4487488646
Iteration 20: objective value = 462547.6199124506
Iteration 30: objective value = 458896.69003507175


([0.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 1.0; 0.0 0.0 … 1.0 0.0; 0.0 1.0 … 0.0 0.0], [2.9828000694528636 1.2204171367551768 … -1.1514563826085844 -1.0308207110916516; 3.0106621399640954 1.2977860722664145 … -1.157062914811369 -1.0534307474036388; 2.978116811851487 1.2589967427517725 … -1.082091790448259 -1.1358451397776328; 3.0093505083151264 1.2875273696465004 … -1.0037683678169946 -1.1605119293080857], ConvergenceHistory("ProxGradGLRM", [Inf, 8.637681448302025e6, 6.854029108531327e6, 5.243988229351777e6, 3.8527715065720095e6, 2.708772851390638e6, 1.8397248807976234e6, 1.2288638433747625e6, 852498.0491710943, 640500.3263282396  …  460876.44186960126, 460322.13001681806, 460081.3390710598, 459747.6050152302, 459512.38438202126, 459290.04356924247, 459040.24689866847, 458896.69003507175, 458778.65373038687, 458696.93631901866], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  

In [22]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm04, nfolds=2, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 4741723 samples and testing on 4743477
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 228060.43770086323
computing train and test error for fold 1:
	train error: 0.04809381023022364
	test error:  0.04851448646387651

forming train and test GLRM for fold 2
training model on 4743477 samples and testing on 4741723
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 230014.1457948012
computing train and test error for fold 2:
	train error: 0.04847290167495832
	test error:  0.04806949338658676


,train_error,test_error
,Float64,Float64
1,0.0480938,0.0485145
2,0.0484729,0.0480695


In [23]:
hingek4 = convert(DataFrame, X04) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [24]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingeparam100.4.csv",  hingek4)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingeparam100.4.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


# k-means with weighted loss funciton

In [54]:
#create an array of positive and negative weighted loss functions
lossesb = Loss[HuberLoss(), HuberLoss(3.233144), OrdinalHingeLoss() ]
lossarr = lossesb 
for i in 3:919
    lossarr = vcat(lossarr, HingeLoss(imp[2][i]) )
print(lossarr)
end

Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.228630900907381, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.228630900907381, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.228630900907381, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:4
└ @ Core ./In[54]:4


3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.228630900907381, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(-0.029199136051568, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(-0.00374605067380904, BoolDomain(), 1.0), WeightedHingeLoss(0.00813799726256531, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDomain(), 1.0), WeightedHingeLoss(-0.00774838025921876, BoolDomain(), 1.0), WeightedHingeLoss(-0.0112564616189466, BoolDomain(), 1.0), WeightedHingeLoss(-0.0428076232524181, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.

(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.401887462963974, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.228630900907381, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(-0.029199136051568, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(-0.00374605067380904, BoolDomain(), 1.0), WeightedH

), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(-0.00374605067380904, BoolDomain(), 1.0), WeightedHingeLoss(0.00813799726256531, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDomain(), 1.0), WeightedHingeLoss(-0.00774838025921876, BoolDomain(), 1.0), WeightedHingeLoss(-0.0112564616189466, BoolDomain(), 1.0), WeightedHingeLoss(-0.0428076232524181, BoolDomain(), 1.0), WeightedHingeLoss(0.00400691717567754, BoolDomain(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.401887462963974, BoolDomain(), 1.0), WeightedHingeLoss(0.000618411540985729, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.002785895108570

...

(), 1.0), WeightedHingeLoss(0.00473346105980751, BoolDomain(), 1.0), WeightedHingeLoss(7.0417859843304e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(-0.000168289069991531, BoolDomain(), 1.0), WeightedHingeLoss(-0.00424430333762854, BoolDomain(), 1.0), WeightedHingeLoss(8.73916457301014e-7, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(-1.48795496805598e-7, BoolDomain(), 1.0), WeightedHingeLoss(5.76522310049651e-5, BoolDomain(), 1.0), WeightedHingeLoss(-0.00368492065957839, BoolDomain(), 1.0), WeightedHingeLoss(-0.00144439551793301, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.000129702494277724, BoolDomain(), 1.0), WeightedHingeLoss(0.000377403049978375, BoolDomain(), 1.0), WeightedHingeLoss(2.03162835480735e-5, BoolDomain(), 1.0), WeightedHingeLoss(-0.00299146522863261, BoolDomain(), 1.0), 

(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(-0.00374605067380904, BoolDomain(), 1.0), WeightedHingeLoss(0.00813799726256531, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDomain(), 1.0), WeightedHingeLoss(-0.00774838025921876, BoolDomain(), 1.0), WeightedHingeLoss(-0.0112564616189466, BoolDomain(), 1.0), WeightedHingeLoss(-0.0428076232524181, BoolDomain(), 1.0), WeightedHingeLoss(0.00400691717567754, BoolDomain(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.401887462963974, BoolDomain(), 1.0), WeightedHingeLoss(0.000618411540985729, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00278589

, BoolDomain(), 1.0), WeightedHingeLoss(0.000137718865596098, BoolDomain(), 1.0), WeightedHingeLoss(0.721097520035912, BoolDomain(), 1.0), WeightedHingeLoss(0.107812725440172, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.000166589421645554, BoolDomain(), 1.0), WeightedHingeLoss(1.04912794747844e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0174248790312528, BoolDomain(), 1.0), WeightedHingeLoss(0.00278118857498796, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00263162959332172, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(1.10478904368101e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(-0.00016375009908857, BoolDomain(), 1.0), WeightedHingeLoss(-8.13002527483895e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDo

Excessive output truncated after 524299 bytes.

), WeightedHingeLoss(0.000129702494277724, BoolDomain(), 1.0), WeightedHingeLoss(0.000377403049978375

In [55]:
size(lossarr)

(920,)

In [ ]:
#k-means with proxgrad, negative and positive loss function weights

In [56]:

k = 3
losses = lossarr
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrmw3 = GLRM(A,losses,rx,ry,k)
paramsw3 = ProxGradParams(inner_iter_Y=40)

ProxGradParams(1.0, 100, 1, 40, 1.0e-5, 0.0001, 0.01)

In [57]:
Xw3,Yw3,chw3 = fit!(glrmw3, paramsw3)

Fitting GLRM
Iteration 10: objective value = -26207.295232421857


([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0], [2.951597625537157 1.2966565298039736 … -0.3348055859959465 0.2454242690984171; 3.010379640895734 1.2966530558066558 … -0.6787459589789004 0.9292044719954764; 3.051966652398445 1.294076838646476 … 0.5845671282599691 -0.261972282668051], ConvergenceHistory("ProxGradGLRM", [Inf, 165660.21193387, 138054.36819950413, 115405.57939350551, 94376.69987073021, 73879.87107454253, 53670.89315422642, 33594.6316029086, 13600.794471166106, -6331.727312218093, -26207.295232421857, -46049.50783599282], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 211.16465592384338, 473.1162037849426, 690.7075889110565, 940.7243688106537, 1191.1135499477386, 1440.3699657917023, 1681.3040916919708, 1921.8877856731415, 2152.50754570961, 2389.050750732422, 2635.134527683258], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0))

In [58]:
Yw3

3×920 Array{Float64,2}:
 2.9516   1.29666  7.03003  -1.00398   …  -1.2985    -0.334806   0.245424
 3.01038  1.29665  6.99917  -0.999977     -0.316703  -0.678746   0.929204
 3.05197  1.29408  7.00052  -1.03554      -0.208416   0.584567  -0.261972

In [62]:
imputeyw3 = impute(losses, Yw3) 
imputeyw3  = convert(DataFrame, imputeyw3) 
CSV.write("../data/iyw3.csv",  imputeyw3)
CSV.read("../data/iyw3.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
,Float64,Float64,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,2.9516,1.29666,7,0,1,1,0,0,1,1,0,0,0
2,3.01038,1.29665,7,0,1,0,0,1,1,1,0,1,0
3,3.05197,1.29408,7,0,1,1,0,0,1,1,0,1,0


In [63]:
Xw3= convert(DataFrame, Xw3) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
CSV.write("../data/Xw3.csv",  Xw3)
CSV.read("../data/Xw3.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#k-means with proxgrad, all loss function weights positive 

In [90]:
lossesb = Loss[HuberLoss(), HuberLoss(3.233144), OrdinalHingeLoss() ]
lossarr = lossesb 
for i in 3:919
    lossarr = vcat(lossarr, HingeLoss(impp[2][i]) )
print(lossarr)
end

Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, Bool

(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672003, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.008137997262565309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0040069171756

(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672003, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.008137997262565309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00400691717567754, BoolDomain(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.40188746296397404, BoolDomain(), 1.0), We

(0.40188746296397404, BoolDomain(), 1.0), WeightedHingeLoss(0.000618411540985729, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00278589510857055, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.9618206972589898e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00387180549143007, BoolDomain(), 1.0), WeightedHingeLoss(0.000215271651905309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(6.428889344341011e-5, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.

(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.11738147927239, BoolDomain(), 1.0), WeightedHingeLoss(4.68020568925754e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.000518249064963447, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672003, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.008137997262565309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDo

...

), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.000129702494277724, BoolDomain(), 1.0), WeightedHingeLoss(0.000377403049978375, BoolDomain(), 1.0), WeightedHingeLoss(2.03162835480735e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0028388393271629397, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.5014172508714302e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.000465702309087595, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00476451996210832, BoolDomain(), 1.0), WeightedHingeLoss(0.0244381813694703, BoolDomain(), 1.0), WeightedHingeLoss(0.0119591569626452, BoolDoma

(0.40188746296397404, BoolDomain(), 1.0), WeightedHingeLoss(0.000618411540985729, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00278589510857055, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.9618206972589898e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00387180549143007, BoolDomain(), 1.0), WeightedHingeLoss(0.000215271651905309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(6.428889344341011e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.11738147927239, BoolDomain(), 1.0), WeightedHingeLoss(4.68020568925754e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.000518249064963447, BoolDomain(), 1.0), WeightedHingeLoss(0.41163263844917797, BoolDomain(), 1.0), WeightedHingeLoss(0.00826408122295437, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), Weigh

Excessive output truncated after 524292 bytes.

(0.0006754823403496389, BoolDomain(), 1.0), WeightedHingeLoss(0.0040535407456485, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00039732309063055297, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00473346105980751, BoolDomain(), 1.0), WeightedHingeLoss(7.0417859843304005e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(8.739164573010141e-7, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(5.76522310049651e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[90]:4
└ @ Core ./In[90]:4


In [91]:
k = 3
losses = lossarr
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrmwp3 = GLRM(A,losses,rx,ry,k)
paramswp3 = ProxGradParams(inner_iter_Y=40)

ProxGradParams(1.0, 100, 1, 40, 1.0e-5, 0.0001, 0.01)

In [92]:
Xwp3,Ywp3,chwp3 = fit!(glrmwp3, paramswp3)

Fitting GLRM
Iteration 10: objective value = 188549.88608233878


([0.0 0.0 … 0.0 0.0; 1.0 0.0 … 1.0 1.0; 0.0 1.0 … 0.0 0.0], [3.0446622632942986 1.3078903311979049 … -1.4038978968660583 -1.008794833091182; 2.966090791115524 1.2815217184312255 … -0.15440441786016945 -0.9464197603457315; 2.9530175794866587 1.2937704395438687 … 0.8863391502622612 0.25991121586410787], ConvergenceHistory("ProxGradGLRM", [Inf, 202132.89444692756, 194692.33866125764, 192247.28541855174, 190791.6918489079, 189986.1342796974, 189491.4666394589, 189146.5065635874, 188903.5075174995, 188702.40212911632, 188549.88608233878, 188421.5126764904, 188316.01564081787, 188229.93583456983], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 187.9382779598236, 407.9282820224762, 659.6310701370239, 889.1935710906982, 1134.242832183838, 1390.4408791065216, 1650.03684091568, 1932.0288038253784, 2165.0231099128723, 2398.8977370262146, 2627.409893989563, 2859.725610971451, 3069.7929

In [93]:
Ywp3

3×920 Array{Float64,2}:
 3.04466  1.30789  7.146    -1.01021  …  -1.00051  -1.4039    -1.00879
 2.96609  1.28152  7.00022  -1.00001     -2.17643  -0.154404  -0.94642
 2.95302  1.29377  7.05797  -1.00095     -1.00129   0.886339   0.259911

In [94]:
imputeywp3 = impute(losses, Ywp3) 
imputeywp3  = convert(DataFrame, imputeywp3) 
CSV.write("../data/iywp3.csv",  imputeywp3)
CSV.read("../data/iywp3.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
,Float64,Float64,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,3.04466,1.30789,7,0,1,0,0,1,0,1,0,1,0
2,2.96609,1.28152,7,0,1,0,0,1,0,1,0,1,0
3,2.95302,1.29377,7,0,1,0,0,1,0,1,0,0,0


In [95]:
Xwp3= convert(DataFrame, Xwp3) 
CSV.write("../data/Xwp3.csv",  Xwp3)
CSV.read("../data/Xwp3.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


# k-means different (non negative)weighted loss functions and proximal gradient descent

In [14]:
function kmeans_weighted(kval, lossarr, matrix)
    k = kval
    losses = lossarr
    A = matrix
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,losses,rx,ry,k)
    params = ProxGradParams(inner_iter_Y=40)

    
    X,Y,ch = fit!(glrm, params) # objective value
    println("X: ")
    println(X)
    println("Y: ")
    println(Y)
    
    imputey = impute(losses, Y) 
    imputey  = convert(DataFrame, imputey) 
    CSV.write(string("../data/iywp", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xwp", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
    return A,X,Y,ch

end    
    



kmeans_weighted (generic function with 1 method)

In [15]:
#loss function array with nonnegative weights
lossesb = Loss[HuberLoss(), HuberLoss(3.233144), OrdinalHingeLoss() ]
lossarr = lossesb 
for i in 3:919
    lossarr = vcat(lossarr, HingeLoss(impp[2][i]) )
print(lossarr)
end

Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, Bool

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[15]:5
└ @ Core ./In[15]:5


), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672003, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.008137997262565309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHi

, BoolDomain(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0)]Loss[HuberLoss(1.0, RealDomain(), 1.0), HuberLoss(3.233144, RealDomain(), 1.0), OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672003, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.008137997262565309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0

1.0, OrdinalDomain(1, 10)), WeightedHingeLoss(0.191690978259395, BoolDomain(), 1.0), WeightedHingeLoss(0.22863090090738103, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0943458778402859, BoolDomain(), 1.0), WeightedHingeLoss(0.00123620398033306, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.17131381206672003, BoolDomain(), 1.0), WeightedHingeLoss(0.204473992997767, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.008137997262565309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0507760291728127, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00400691717567754, BoolDomain(), 1.0), WeightedHingeLoss(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDo

...

, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.11738147927239, BoolDomain(), 1.0), WeightedHingeLoss(4.68020568925754e-6, BoolDomain(), 1.0), WeightedHingeLoss(0.000518249064963447, BoolDomain(), 1.0), WeightedHingeLoss(0.41163263844917797, BoolDomain(), 1.0), WeightedHingeLoss(0.00826408122295437, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(1.43200527868604e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0027550029022820896, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.074603442799225, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.014616652111906

), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(5.76522310049651e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.000129702494277724, BoolDomain(), 1.0), WeightedHingeLoss(0.000377403049978375, BoolDomain(), 1.0), WeightedHingeLoss(2.03162835480735e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0028388393271629397, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.5014172508714302e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.000465702309087595, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(

(0.0996021174217312, BoolDomain(), 1.0), WeightedHingeLoss(0.0198447185528233, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.40188746296397404, BoolDomain(), 1.0), WeightedHingeLoss(0.000618411540985729, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00278589510857055, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.9618206972589898e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.00387180549143007, BoolDomain(), 1.0), WeightedHingeLoss(0.000215271651905309, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(6.428889344341011e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.11738147927239, BoolDomain(), 1.0

Excessive output truncated after 524292 bytes.

(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(2.9597685682163502e-5, BoolDomain(), 1.0), WeightedHingeLoss(0.0, BoolDomain(), 1.0), WeightedHingeLoss(0.0006754823403496389, BoolDomain(), 1.0),

In [ ]:
kmeans_weighted(3, lossarr, df3)

In [62]:
kmeans_weighted(4, lossarr, df3)

Fitting GLRM
Iteration 10: objective value = 188675.43251117054
X: 


[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0

 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0

 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0]
Y: 
[3.0653996362602913 1.2497975803549024 7.005646728604953 -0.9999913486462259 1.0008152968412052 0.05159822161508028 -1.062104106665735 -1.0016510050562046 -0.08120040518769228 1.0000978730393288 -1.0004306665372509 -0.1914930719792384 -1.0007392392572063 0.33661883485536803 -0.5853404258285012 -0.09117575416232251 1.3015172284922898 1.2135558621441631 -1.0002108103766807 -1.0013835613401474 0.8107615306347946 -0.2496890911773429 -2.2303677283675953 -0.8954592125149348 -0.78209974204434

"../data/Xwp4.csv"

In [16]:
kmeans_weighted(5, lossarr, df3)

Fitting GLRM
Iteration 10: objective value = 188838.52237238648
X: 


[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0

 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 1.0

 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0

...

 0.02754074515936939 0.4039236341689124 0.42781325174371515 -0.43513278233946856 0.21724795351233098 -1.2139780588877702 0.037398605121555656 1.1459827460256877 -2.285225792935493 0.310419595400684 -1.0190224574889482 -1.0174926798695958 0.34375227613936116 1.4166377167105484 -1.0007161732881205 -1.003308515890027 -1.002921439024262 0.11554379828836081 0.9608618252802706 -1.0020507279450022 -0.11799092060835588 -1.1182033044589783 -1.296226017363191 -0.7144965184242325 -1.0029358423034405 -0.0787386097090249 -0.43705524690473485 -1.4590653709807304 -0.5456697341395201 1.179632024515501 0.20645058984772482 -0.7322815313691106 0.3257936046644992 1.6476822446870436 -0.42114810344647724 1.0999536201974172 -0.6893296780904412 0.7730887019958481 0.9177206533394692 1.213265526296371 0.7355558425591276 0.5498024425511998 0.5538827212060681 -0.5574638067919006 -1.0291775250012025 0.159525101397172 0.9316899780614871 -0.2830976164902313 -2.3880378590882962 -0.9197629191676459 -0.2960920648598668

 -1.0140774523368072 -1.0000000102882303 -1.9948630176078879 -1.0120216394829689 -1.0034180819682303 -1.0014623479431943 -1.0072805089347017 1.7728155740386233 -0.11050637415568319 -0.12747648923689855 -0.6055852242562213 -0.9434133170176046 -1.0024569061015756 -0.4804645068845836 -0.7782726894664805 0.47090537686104744 -0.9750339083784307 -1.0000116252053148 -1.0000232405531306 -1.0020480624386645 0.4044453164817231 -0.22326625498802677 -0.42781410848227835 -1.028986673454229 -0.2826234769278144 -1.3302120735321898 0.5965415676632746 1.4719010225728477 0.04860906363206201 -0.6017150638779949 -1.0032732949736645 -0.3090197094439058 -1.1953757947772405 -0.9999997654558509 -1.2224317796653816 1.184181730920837 -1.0843329366528263 -1.0000986154431786 -1.0000007715469392 0.6302308392633148 0.40015916981471905 0.22026822246029376 2.6510088227840063 -1.0199683432229507 -2.2497409679583695 -1.0233771404380747 -0.9999959347451206 -0.9999977145376616 -0.9999990874400887 -0.48022091896399305 -0.

 -0.41830166062202995 1.3915291306530195 -0.8731515017845265 1.0152927876985915 -0.2295245343131461 0.048405027628797455 1.9162813273953367 0.4479006499996499 -0.11655636126196633 -0.6546971270758302; 3.068509513472207 1.2662517231983073 7.0042655732784285 -1.0001368002060231 1.000319418284625 0.1804318304377234 -1.026257631816175 -1.1231543712532974 0.609568763941497 0.9999499309824504 -1.0000346356916123 1.2478976487917912 -1.0001507254696431 0.2753581014574445 -0.9841700730228348 -0.15517445070113817 -0.06859081175848729 0.520283715227394 -1.0012198646639108 -1.0052808840340726 0.9999994877125041 -0.5270961833405365 0.6311934261918085 0.5201482861507973 -1.8791932095384987 -1.0060577146932725 0.2424584144199629 0.7713270860492762 -0.8404825922952631 -0.37251281904970257 0.9278770539971565 1.180189720919946 -1.0017609753305696 0.04366917320060891 0.4110940002955831 -0.3846537565599277 1.1143918857192772 -0.406860381268817 -1.0064661320200583 0.8145154894443006 -1.000070777076242 -1.0

 0.014998444360591126 -0.6528372851688122 0.011101979503160714 1.0252166421515763 1.5279450009074163 -0.9986837752910842 -1.0074280876317991 0.9028133403373573 -1.0009462854182323 0.8532163965972376 -1.2545807289449922 0.5046942784371629 -1.394598293389053 0.707442020331868 -1.0000000027203304 -1.0009498691224925 -1.0024099002233091 0.2727777850427454 2.823803213532506 0.32404372860514197 -0.35138839958633084 -2.108787295263565 0.8445413363986548 -1.5784192977529496 -1.0014425040113901 -1.0000002274569746 -0.9405193382354343 -1.0074222701947106 1.5555401649578011 -0.26029391609476593 -1.227161425541295 0.24530480492132084 -1.0022538759241613 0.23276491631052107 -2.3646525092117616 -1.1955535963391466 -0.43241794154662316 -0.5008535440179046 0.5340956328687454 -0.9811187635920491 -0.42661315553430723 1.8084092512366676 0.12676209272247008 -2.756265181231597 0.08868441150378967 -1.001039321342614 -1.0034627300924523 0.6024802312754061 -1.4254451212315353 -0.16378367603575028 -1.679796802

(10310×920 DataFrame. Omitted printing of 915 columns
│ Row   │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│       │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├───────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1     │ 3.666   │ 1               │ 4           │ 0        │ 1         │
│ 2     │ 1.2885  │ 1               │ 6           │ 0        │ 1         │
│ 3     │ 1.0508  │ 1               │ 8           │ 1        │ 0         │
│ 4     │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 5     │ 0.5124  │ 1               │ 4           │ 0        │ 1         │
│ 6     │ 3.5466  │ 1               │ 6           │ 0        │ 1         │
│ 7     │ 1.1224  │ 1               │ 8           │ 1        │ 0         │
│ 8     │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 9     │ 1.1397  │ 1               │ 7           │ 0        │ 1         │
│ 10    │ 1.9138  │ 1               │ 7       

In [17]:
kmeans_weighted(6, lossarr, df3)

Fitting GLRM
Iteration 10: objective value = 188877.36855883256
X: 


[0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0

 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0

 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0

...

 -1.0817132493703387 -0.576145551976966 -1.0216684142636463 -1.0000373373864384 0.7692267317485574 -1.00467237286614 -1.054846814022302 -1.0023737922453515 0.5543563656432379 0.0648536963538481 -0.9029870163769893 0.6907038566111584 0.5847337935136923 0.13351451373408516 0.4841254167910607 0.9495733156249376 0.5522737512246965 -1.0015565042267396 -1.0015669255281974 -1.0000000036634002 0.029782420644884584 -1.04483047658872 -1.1417700073750074 1.1068778227380593 -1.0101634120090843 0.39409959373070114 -0.5401583164479172 -0.21259278639177967 0.7066521065413394 -1.0063746174955048 0.19714121058848147 -0.9999995468925711 0.8107720952528154 -1.0000000015272827 1.6365742809193666 1.113857526601826 0.3269983643667888 -1.0106972618251524 1.723990453297078 -2.2429917683241123 -2.290874917407336 -1.0465421809360667 -0.9999999774549613 -0.031615423374241336 1.3303390122433272 1.7635019835160284 -0.32273029379606866 -0.8306082316089111 -1.0002158443100984 -1.0013492127243258 -0.999998524649535 1

 -1.0013403367267595 -1.0001534198294577 -1.0244512883296204 -1.0335253489162393 -1.14458632200419 -0.7205373573431115 0.31852066638052334 1.2648224492197104 -0.8543723711478673 -1.0033557745600472 -2.0717570079432015 -0.9812718828573861 -1.108016257365499 0.790089506015313 1.47526405478898 -0.2731402499763023 1.1478687936691296 1.0487701023291416 0.8965941486452215 -0.8995614254981333 0.9157463467932042 -1.3201295712781358 0.34394935393349824 1.7397114645143452 -0.5871834704230502 -0.24417372041269675 0.5326809106769999 0.3383530294024113 -1.3154866170108672 1.477039233623574 0.5112762021622511 0.8941082572965281 -1.0032030382618409 -0.750221908306745 -1.0546420787051636 -0.021534448436885924 -1.0004810627831653 -1.000915501281558 -0.999999990625789 0.9733490746853721 -0.4168209981052449 -0.4896725273208921 -0.44868214009602747 -0.994252449419601 -1.0000002694573458 0.33934546364516066 0.7778306179089245 1.269419019428204 -0.9750670072487252 0.3008974355419831 -0.6032703693260074 0.04

 -0.5173605431023365 1.1090029753435022 -1.0281599580924117 -1.6396755325413954 -1.2425682908959228 -1.1627473146069358 1.2010997190918695 0.33631473464752154 -1.704553762017706 -1.000000052423416 -1.0102040248427315 0.6380363531465093 -0.8767203084613704 -1.8449175930480939 -1.402148645331851 -1.002334268332885 -1.0062696233375534 -1.0000000430964195 1.0519142310195762 -0.130396180804028 -1.841632119984892 -0.21456783140079047 -1.0000003463657836 0.9033332464250221 1.3294352712343802 -1.0058701261664487 -1.3097033117216554 2.6295518800835835 -1.0000009520207631 -1.00352015799722 -0.9999999978880901 -1.0864768217597824 -1.000000041854185 -1.0041055464099038 -1.4239046206587724 -1.0030770147138286 -1.5060595551385945 -1.0011241292045474 -0.3578431015052614 -1.0000027843931834 -1.9460251901037486 -1.0000010991306651 -0.7798008217492528 1.886310609159508 -0.9999993493403051 0.05588252104758214 -0.07725057070655332 -0.729491500234569 0.13713401857996074 -1.0225998417273856 0.60501828472377

 -0.06594118276739558 -0.8377171347876728 -0.7872935508315851 0.7422983855375219 1.1618117902914051 -1.6943034647798183 0.40360187048652185 0.12165093779175358 -0.1553451164777572 0.8396276127864077 0.07144114699676464 -0.367588896015217 0.9306301219189925 -2.0063030427454147 -0.6079793971344739 0.2510753344061979 0.980545834901514 -1.3016744029263574 -1.2449829742337715 0.12004202754425898 -0.5706615259728133 -1.305771929214758 0.9680530667500288 0.3698306514819778 -1.0115401550370313 -1.048383341969427 -0.9323715396415031 1.061081980460601 -1.0090836383795887 -1.0000000242318936 -1.0000000802178457 -0.3787369014705909 1.152295096481825 -1.0054169425715829 -2.980678335013036 -1.337832006296242 -0.8234520598342161 -1.3207607654460551 -1.0026094183995995 1.0397825163874017 0.606545587615761 -1.5165476914183604 0.4563257745201107 0.4275328085163185 -1.9838173688437821 0.8810706621361893 -0.5870093773653917 -1.3042972560728476 -0.9332884527628021 0.0422101000963591 0.7280524864577095 -1.4

(10310×920 DataFrame. Omitted printing of 915 columns
│ Row   │ LOS     │ HOSPITALIZATION │ ORDINAL_AGE │ ELECTIVE │ EMERGENCY │
│       │ Float64 │ Int64           │ Int64       │ Int64    │ Int64     │
├───────┼─────────┼─────────────────┼─────────────┼──────────┼───────────┤
│ 1     │ 3.666   │ 1               │ 4           │ 0        │ 1         │
│ 2     │ 1.2885  │ 1               │ 6           │ 0        │ 1         │
│ 3     │ 1.0508  │ 1               │ 8           │ 1        │ 0         │
│ 4     │ 14.2664 │ 2               │ 9           │ 0        │ 1         │
│ 5     │ 0.5124  │ 1               │ 4           │ 0        │ 1         │
│ 6     │ 3.5466  │ 1               │ 6           │ 0        │ 1         │
│ 7     │ 1.1224  │ 1               │ 8           │ 1        │ 0         │
│ 8     │ 5.3757  │ 1               │ 7           │ 1        │ 0         │
│ 9     │ 1.1397  │ 1               │ 7           │ 0        │ 1         │
│ 10    │ 1.9138  │ 1               │ 7       

In [ ]:
kmeans_quadloss_drg(3, diabetesdrg)

In [ ]:
# positive weights * 3 

# GLRM k-means loss function with positive weight hypertension

In [ ]:
#kmeans loss function not weighted

In [84]:
hypernan = CSV.read("../data/hypertension_nonan.csv")
hypernan = select!(hypernan, Not("Unnamed: 0"))
hypernan = select!(hypernan, Not(:SUBJECT_ID )) 
hypernan

,003,004,005,007,008,009,010,011,012,013,014,015
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
for name in names(hypernan)
       print(":", name, ", ")
end

:003, :004, :005, :007, :008, :009, :010, :011, :012, :013, :014, :015, :018, :023, :027, :031, :032, :033, :034, :035, :036, :038, :039, :040, :041, :042, :046, :047, :048, :049, :052, :053, :054, :057, :058, :070, :075, :077, :078, :079, :088, :091, :093, :094, :097, :099, :110, :111, :112, :115, :117, :121, :122, :123, :127, :130, :131, :132, :133, :135, :136, :137, :138, :139, :140, :141, :142, :143, :144, :145, :146, :147, :148, :149, :150, :151, :152, :153, :154, :155, :156, :157, :158, :159, :160, :161, :162, :163, :164, :170, :171, :172, :173, :174, :175, :176, :179, :180, :182, :183, :184, :185, :187, :188, :189, :191, :192, :193, :194, :195, :196, :197, :198, :199, :200, :201, :202, :203, :204, :205, :206, :207, :208, :209, :210, :211, :212, :213, :214, :215, :216, :217, :218, :220, :221, :223, :225, :226, :227, :228, :229, :230, :232, :233, :235, :236, :237, :238, :239, :240, :241, :242, :244, :245, :246, :249, :250, :251, :252, :253, :254, :255, :256, :257, :258, :259, :260

In [85]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 1003)
#loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss2; loss1]
losses

1005-element Array{Loss,1}:
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0,

In [89]:
k = 3
losses = QuadLoss()

A= hypernan
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm640 = GLRM(A,losses,rx,ry,k)
params640 = ProxGradParams(inner_iter_Y=40)


ProxGradParams(1.0, 100, 1, 40, 1.0e-5, 0.0001, 0.01)

In [90]:
X640,Y640,ch640 = fit!(glrm640, params640)

Fitting GLRM
Iteration 10: objective value = 1.8698987960285519e6


([1.0 1.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0], [0.00016291951329514963 1.8073314825610374e-11 … 1.2803844896112757 5.988621195214561; 0.00016257519181027443 1.4385440385274992e-12 … 1.2762152494881398 5.878761728889956; -1.477957067973571e-12 0.00017415538744531067 … 1.2589689910059334 5.318109236371424], ConvergenceHistory("ProxGradGLRM", [Inf, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6, 1.8698987960285519e6], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 393.35517501831055, 833.6293449401855, 1318.6259548664093, 1795.261855840683, 2206.5278816223145, 2630.6129586696625, 3180.1887605190277, 3910.349247455597, 4607.6868505477905, 5297.4420845508575, 5730.03617143631], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [91]:
X640 = convert(DataFrame, X640) 
CSV.write("/Users/kathy908000/github/health-care-research/data/Xhyper3.csv",  X640)
CSV.read("/Users/kathy908000/github/health-care-research/data/Xhyper3.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


In [92]:
imputey640 = impute(losses, Y640) 
CSV.write("/Users/kathy908000/github/health-care-research/data/iyhyper3.csv",  imputey640)
CSV.read("/Users/kathy908000/github/health-care-research/data/iyhyper3.csv")

MethodError: MethodError: no method matching impute(::QuadLoss, ::Array{Float64,2})
Closest candidates are:
  impute(::Loss, !Matched::Float64) at /Users/kathy908000/.julia/packages/LowRankModels/uhtB9/src/impute_and_err.jl:37
  impute(!Matched::Array{LossSubtype,1}, ::Array{Float64,2}) where LossSubtype<:Loss at /Users/kathy908000/.julia/packages/LowRankModels/uhtB9/src/impute_and_err.jl:166

In [ ]:
#loss function array with nonnegative weights
lossesb = Loss[HuberLoss(), HuberLoss(3.233144) ]
lossarr = lossesb 
for i in 3:919
    lossarr = vcat(lossarr, HingeLoss(impp[2][i]) )
print(lossarr)
end